<a href="https://colab.research.google.com/github/alexchen1999/covid-19-sample-strain-classification/blob/main/random_sampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Bio
!pip install bcbio-gff
!pip install ncbi-acc-download

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 270 kB 5.2 MB/s 
     |████████████████████████████████| 2.3 MB 39.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 44 kB 2.0 MB/s 
  Created wheel for bcbio-gff: filename=bcbio_gff-0.6.9-py3-none-any.whl size=15954 sha256=93867b13d582c02e5fefc647d369cb434c76e2e042d0adbd0162ccf406c309b5
  Stored in directory: /root/.cache/pip/wheels/fb/fc/e0/2e7658046d32b794fdfd1e0ec266dc8bae21c1811b18951b6d
Successfully built bcbio-gff
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
import json
import numpy as np
import pandas as pd
import subprocess

from BCBio import GFF
from Bio import Align, SeqIO
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import shuffle

In [3]:
covid_strains = ['B.1.1.7_sequences.csv', 'P.1_sequences.csv', 'B.1.617.2_sequences.csv', 'BA.1.1_sequences.csv']

In [4]:
alpha = pd.read_csv(covid_strains[0])
gamma = pd.read_csv(covid_strains[1])
delta = pd.read_csv(covid_strains[2])
omicron = pd.read_csv(covid_strains[3])

In [5]:
shortest = min([alpha.shape[0], gamma.shape[0], delta.shape[0], omicron.shape[0]])
print(shortest)

3735


In [6]:
alpha_sample = alpha.sample(n=shortest)
print(alpha_sample.shape[0])
gamma_sample = gamma.sample(n=shortest)
print(gamma_sample.shape[0])
delta_sample = delta.sample(n=shortest)
print(delta_sample.shape[0])
omicron_sample = omicron.sample(n=shortest)
print(omicron_sample.shape[0])

3735
3735
3735
3735


In [7]:
print(alpha_sample['Country'].unique())
print(gamma_sample['Country'].unique())
print(delta_sample['Country'].unique())
print(omicron_sample['Country'].unique())

['USA' 'Iraq' 'Japan' 'Pakistan' 'Finland' 'Nigeria' 'Austria' 'Djibouti'
 'Spain' 'India' 'Egypt' 'Philippines' 'West Bank' 'Bangladesh' 'Denmark'
 'Mexico' 'Italy']
['USA' 'Dominican Republic' 'Brazil' 'Paraguay' 'Mexico' 'Taiwan' 'Chile'
 'Italy' 'Peru']
['USA' 'Bahrain' 'Bangladesh' 'Egypt' 'Japan' 'Mongolia' 'India' 'Myanmar'
 'Pakistan' 'Chile' 'Uzbekistan' 'Gabon' 'Russia' 'Denmark' 'Jamaica'
 'West Bank' 'China']
['USA' 'Bahrain' 'West Bank' 'France' 'Japan']


In [8]:
# From SARS-Cov-2 GFF file, find genomic location of Spike Glycoprotein
in_file = "GCF_009858895.2_ASM985889v3_genomic.gff"
in_handle = open(in_file)
features = []
for rec in GFF.parse(in_handle):
    features = rec.features
in_handle.close()

start = -1
end = -1

# spike glycoprotein locus tag GU280_gp02
for i in range(len(features)):
    if "GU280_gp02" in features[i].id:
      print(features[i])
      print(features[i].location)
      print(int(features[i].location._start))
      print(int(features[i].location._end))
      start = int(features[i].location._start)
      end = int(features[i].location._end)

type: gene
location: [21562:25384](+)
id: gene-GU280_gp02
qualifiers:
    Key: Dbxref, Value: ['GeneID:43740568']
    Key: ID, Value: ['gene-GU280_gp02']
    Key: Name, Value: ['S']
    Key: gbkey, Value: ['Gene']
    Key: gene, Value: ['S']
    Key: gene_biotype, Value: ['protein_coding']
    Key: gene_synonym, Value: ['spike glycoprotein']
    Key: locus_tag, Value: ['GU280_gp02']
    Key: source, Value: ['RefSeq']

[21562:25384](+)
21562
25384


In [9]:
# Find spike glycoprotein sequence from reference genome
record = SeqIO.read("NC_045512.2.fa", "fasta")

# Take from 21562 to 25384
spike_seq = str(record._seq)[start:end]
print(spike_seq)
print(len(spike_seq))

ATGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGTGTTAATCTTACAACCAGAACTCAATTACCCCCTGCATACACTAATTCTTTCACACGTGGTGTTTATTACCCTGACAAAGTTTTCAGATCCTCAGTTTTACATTCAACTCAGGACTTGTTCTTACCTTTCTTTTCCAATGTTACTTGGTTCCATGCTATACATGTCTCTGGGACCAATGGTACTAAGAGGTTTGATAACCCTGTCCTACCATTTAATGATGGTGTTTATTTTGCTTCCACTGAGAAGTCTAACATAATAAGAGGCTGGATTTTTGGTACTACTTTAGATTCGAAGACCCAGTCCCTACTTATTGTTAATAACGCTACTAATGTTGTTATTAAAGTCTGTGAATTTCAATTTTGTAATGATCCATTTTTGGGTGTTTATTACCACAAAAACAACAAAAGTTGGATGGAAAGTGAGTTCAGAGTTTATTCTAGTGCGAATAATTGCACTTTTGAATATGTCTCTCAGCCTTTTCTTATGGACCTTGAAGGAAAACAGGGTAATTTCAAAAATCTTAGGGAATTTGTGTTTAAGAATATTGATGGTTATTTTAAAATATATTCTAAGCACACGCCTATTAATTTAGTGCGTGATCTCCCTCAGGGTTTTTCGGCTTTAGAACCATTGGTAGATTTGCCAATAGGTATTAACATCACTAGGTTTCAAACTTTACTTGCTTTACATAGAAGTTATTTGACTCCTGGTGATTCTTCTTCAGGTTGGACAGCTGGTGCTGCAGCTTATTATGTGGGTTATCTTCAACCTAGGACTTTTCTATTAAAATATAATGAAAATGGAACCATTACAGATGCTGTAGACTGTGCACTTGACCCTCTCTCAGAAACAAAGTGTACGTTGAAATCCTTCACTGTAGAAAAAGGAATCTATCAAACTTCTAACTTTAGAGTCCAACCAACAGAATCTATTGTTAGATTTCCTAATATTACAA

In [10]:
alpha_sample_accessions = []
for i in range(shortest):
  alpha_sample_accessions.append(alpha_sample.iloc[i]['Accession'])

In [11]:
!ncbi-acc-download --help

usage: ncbi-acc-download [-h] [-m {nucleotide,protein}] [--api-key API_KEY]
                         [-e {none,loads,all,correct}]
                         [-F {fasta,genbank,featuretable,gff3}] [-o OUT]
                         [-p PREFIX] [-g RANGE] [-r] [--url] [-v]
                         NCBI-accession [NCBI-accession ...]

positional arguments:
  NCBI-accession

optional arguments:
  -h, --help            show this help message and exit
  -m {nucleotide,protein}, --molecule {nucleotide,protein}
                        Molecule type to download. Default: nucleotide
  --api-key API_KEY     Specify USER NCBI API key. More info at
                        https://www.ncbi.nlm.nih.gov/books/NBK25497/
  -e {none,loads,all,correct}, --extended-validation {none,loads,all,correct}
                        Perform extended validation. Possible options are
                        'none' to skip validation, 'loads' to check if the
                        sequence file loads in Biopython, or '

In [12]:
# Test NCBI accession download
test = alpha_sample_accessions[0]
print(test)

MZ164127.1


In [ ]:
!ncbi-acc-download --format fasta $test

In [ ]:
for i in range(3):
  subprocess.run(["ncbi-acc-download", "--format", "fasta", alpha_sample_accessions[i]])

In [13]:
# SeqIO
record = SeqIO.read("MZ039190.1.fa", "fasta")
seq = str(record._seq)[start:end]

FileNotFoundError: ignored

In [14]:
# Pairwise alignment
aligner = Align.PairwiseAligner()
aligner.mode = 'global'
alignments = aligner.align(seq, spike_seq)

NameError: ignored

In [16]:
# Alpha sample accessions
# Take from 21562 to 25384 from samples, should be a good enough approximation
# Credit to https://2-bitbio.com/2018/06/one-hot-encode-dna-sequence-using.html

alpha_spike_seq_one_hot = pd.DataFrame(columns=["Accession", "Seq"])
alpha_spike_seq_df = pd.DataFrame(columns = ['Accession', 'Seq'])
for i in range(3):
  accession = alpha_sample_accessions[i]
  subprocess.run(["ncbi-acc-download", "--format", "fasta", accession])
  record = SeqIO.read(accession + ".fa", "fasta")
  spike_seq = str(record._seq)[start:end]

  # One-hot encoding
  seq_array = array(list(spike_seq))
  label_encoder = LabelEncoder()
  integer_encoded_seq = label_encoder.fit_transform(seq_array)
    
  # one hot the sequence
  onehot_encoder = OneHotEncoder(sparse=False)
  # reshape because that's what OneHotEncoder likes
  integer_encoded_seq = integer_encoded_seq.reshape(len(integer_encoded_seq), 1)
  onehot_encoded_seq = onehot_encoder.fit_transform(integer_encoded_seq)

  # convert the sequence from ndarray to json string to simplify conversion
  onehot_encoded_seq = json.dumps(onehot_encoded_seq.tolist())

  alpha_spike_seq_df.loc[len(alpha_spike_seq_df.index)] = [accession, onehot_encoded_seq]
  subprocess.run(['rm', accession + ".fa"])

In [23]:
# All
dfs = [alpha_sample, delta_sample, gamma_sample, omicron_sample]
labels = ['Alpha', 'Delta', 'Gamma', 'Omicron']
spike_seq_df = pd.DataFrame(columns = ['Accession', 'Seq', 'Label'])
num_samples = 1000
for i in range(num_samples):
  for j in range(4):
    accession = dfs[j].iloc[i]['Accession']

    subprocess.run(["ncbi-acc-download", "--format", "fasta", accession])
    record = SeqIO.read(accession + ".fa", "fasta")
    spike_seq = str(record._seq)[start:end]

    # One-hot encoding
    seq_array = array(list(spike_seq))
    label_encoder = LabelEncoder()
    integer_encoded_seq = label_encoder.fit_transform(seq_array)
    
    #one hot the sequence
    onehot_encoder = OneHotEncoder(sparse=False)
    #reshape because that's what OneHotEncoder likes
    integer_encoded_seq = integer_encoded_seq.reshape(len(integer_encoded_seq), 1)
    onehot_encoded_seq = onehot_encoder.fit_transform(integer_encoded_seq)

    # convert the sequence from ndarray to json string to simplify conversion
    onehot_encoded_seq = json.dumps(onehot_encoded_seq.tolist())

    spike_seq_df.loc[len(spike_seq_df.index)] = [accession, onehot_encoded_seq, labels[j]]
    subprocess.run(['rm', accession + ".fa"])
  print((i+1) / num_samples * 100, "% complete")

df = shuffle(spike_seq_df)
df.to_csv('spike_seq_one_hots.csv')

0.1 % complete
0.2 % complete
0.3 % complete
0.4 % complete
0.5 % complete
0.6 % complete
0.7000000000000001 % complete
0.8 % complete
0.8999999999999999 % complete
1.0 % complete
1.0999999999999999 % complete
1.2 % complete
1.3 % complete
1.4000000000000001 % complete
1.5 % complete
1.6 % complete
1.7000000000000002 % complete
1.7999999999999998 % complete
1.9 % complete
2.0 % complete
2.1 % complete
2.1999999999999997 % complete
2.3 % complete
2.4 % complete
2.5 % complete
2.6 % complete
2.7 % complete
2.8000000000000003 % complete
2.9000000000000004 % complete
3.0 % complete
3.1 % complete
3.2 % complete
3.3000000000000003 % complete
3.4000000000000004 % complete
3.5000000000000004 % complete
3.5999999999999996 % complete
3.6999999999999997 % complete
3.8 % complete
3.9 % complete
4.0 % complete
4.1000000000000005 % complete
4.2 % complete
4.3 % complete
4.3999999999999995 % complete
4.5 % complete
4.6 % complete
4.7 % complete
4.8 % complete
4.9 % complete
5.0 % complete
5.1 % comp

In [21]:
# Example ndarray conversion:

a = np.array([[0, 1, 0, 0], [1, 0, 0, 0]])
print(a)
print(type(a))
print(a.shape)
b = a.tolist()
s = json.dumps(b)
arr = np.array(json.loads(s))
print(arr)
print(type(arr))
print(arr.shape)

[[0 1 0 0]
 [1 0 0 0]]
<class 'numpy.ndarray'>
(2, 4)
[[0 1 0 0]
 [1 0 0 0]]
<class 'numpy.ndarray'>
(2, 4)


In [ ]:
# Final check that df is well-formed
df
print(len(df.iloc[0]['Seq']))

3822
